In [1]:
%load_ext autoreload
%autoreload 2

In [176]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from functools import partial

import numpy as np
import pandas as pd
import ipyvuetify as v
import win32com.client as win32
from pandas.tseries.offsets import BDay
from scipy import stats

from gioutils import giobhutils as gio
from gioutils.ezutils import SQL
from gioutils.utils import parse_offset, today
from gioutils.gui.pandas import millifyp as millify
from gioutils.pretty_html_table import build_table

from api.risk.var import VaRPnl
from api.email import smtp_server, to_email_test, to_email_all, df_to_html, from_email, to_email_ext

# Email

# Limits

capital = 300e6
var_flag = 1.5 / 100
var_standalone_limit = 25 / 100
var_limit = capital * var_flag
daily_tail_risk_limit = 3.5 / 100
weekly_tail_risk_limit = 3.5 / 100
correlation_limit = 66 / 100
position_counts_limit = 10
send_email=True
book = 'MU'
capital = 300e6 if book == 'MM' else 200e6

category_exclusion = [
    "SCF"
]
strategy_exclusion = [
    'CASH_USAGE'
]

strategy_counts_limit = {
    "RATES": 8,
    "FX": 3,
    'EQD': 1,
}
exclude_cash = True
daily_period = "2y"
weekly_period = "5y"


def fill_limit(df, col1, col2):
    c1 = 'background-color: red'
    c2 = 'background-color: lime'
    df_new = pd.DataFrame('', index=df.index, columns=df.columns)
    df_new.loc[df[col1] > df[col2], col1] = c1
    df_new.loc[df[col1] <= df[col2], col1] = c2

    return df_new


# VaR


dv = VaRPnl()
dv.get_data(
    offset=daily_period,
    exclude_cash=False,
    exclude_null_notional=True,
    book=book,
)

wv = VaRPnl()
wv.get_data(
    offset=weekly_period,
    exclude_cash=False,
    horizon='5d',
    book=book
)

var_dollar = dv.var()
var_check = pd.DataFrame(
    [
        [- var_dollar, var_limit],
        [- var_dollar / capital * 100, var_flag * 100],
        [- var_dollar / var_limit * 100, 100]

    ],
    index=['var_dollar', 'var_pct', 'var_utilization'],
    columns=['value', 'limit'],
).rename_axis('measure')
# var_check.style.apply(partial(fill_limit, col1='value', col2='limit'), axis=None).format('{:.2f}')

ref = dv.ref.loc[~dv.ref["positionName"].str.contains("CASH")] if exclude_cash else dv.ref
ref = ref
hist = dv.hist.loc[:, pd.MultiIndex.from_frame(ref[['strategy', 'positionName']].drop_duplicates())]
position_counts = (
    ref.query(f"riskCategory != {category_exclusion}").groupby(["strategy", "riskCategory", "positionName2"])
    .count()
    .groupby(level=[0, 1])["positionName"]
    .count()
    .rename("count")
)

options_count = (
    ref.query(f"riskCategory != {category_exclusion} and tradeGroup == 'OPT'").groupby(
        ["strategy", "riskCategory", "positionName2"])
    .count()
    .groupby(level=[0, 1])["positionName"]
    .count()
    .rename("count")
)

position_counts = (position_counts - (options_count - 1)).fillna(position_counts)

strategy_counts = (
    position_counts.reset_index()
    .groupby("riskCategory")["strategy"]
    .count()
    .rename("count")
)

### Number of strategies in each category

strategy_check = pd.concat(
    [
        strategy_counts,
        pd.Series(strategy_counts_limit, name="limit").rename_axis(
            strategy_counts.index.name
        ),
    ],
    axis=1,
).fillna(0)
# strategy_check.style.apply(partial(fill_limit, col1='count', col2='limit'), axis=None).format('{:.2f}')

### Number of positions in each Strategy

position_check = position_counts.to_frame().assign(limit=position_counts_limit)
# position_check.style.apply(partial(fill_limit, col1='count', col2='limit'), axis=None).format('{:.2f}')

### Standalone VaR

standalone_var_check = (
            - hist.groupby(axis=1, level=0, sort=False).sum().iloc[-522:].quantile(0.05, interpolation='weibull') / var_limit * 100) \
    .rename('value').to_frame().rename_axis('strategy').assign(limit=var_standalone_limit * 100)
# standalone_var_check.style.apply(partial(fill_limit, col1='value', col2='limit'), axis=None).format('{:.2f}')

### Tail Risk
weekly_hist_by_strat_df =  wv.hist.groupby(level=0, axis=1).sum()

daily_tail_risk_check = (- dv.hist.sum(axis=1).nsmallest(1) / capital * 100).rename('value').to_frame().assign(
    limit=daily_tail_risk_limit * 100).rename_axis('daily')

weekly_tail_risk_check = (- wv.hist.sum(axis=1).nsmallest(1) / capital * 100).rename('value').to_frame().assign(
    limit=weekly_tail_risk_limit * 100).rename_axis('weekly')

tail_risk_check = pd.concat([daily_tail_risk_check, weekly_tail_risk_check], keys=['daily', 'weekly'])

daily_tail_risk_by_strategy = - dv.hist.groupby(level=0, axis=1).sum()\
    .apply(lambda x, n: pd.Series.nsmallest(x, n=n).squeeze(), axis=0, n=1).rename('daily')

weekly_tail_risk_by_strategy = - weekly_hist_by_strat_df \
    .apply(lambda x, n: pd.Series.nsmallest(x, n=n).squeeze(), axis=0, n=1).rename('weekly')

tail_risk_by_strategy = pd.concat([daily_tail_risk_by_strategy, weekly_tail_risk_by_strategy], axis=1).dropna() / capital * 100

res = {}
for col in weekly_hist_by_strat_df:
        res[col] = weekly_hist_by_strat_df.drop(col, axis=1).sum(axis=1).nsmallest(1).rename('marginal_tail').to_frame()
weekly_marginal_tail_risk_by_strat =  - ((- pd.concat(res) / capital * 100) - weekly_tail_risk_check['value'].squeeze()).sort_values('marginal_tail').rename_axis(['strategy', 'date'])


### Correlation
corr = {}
g = hist.groupby(level=0, axis=1)
if len(g) > 1:
    for index, group in g:
        strategy = group.sum(axis=1)
        ptf = hist.drop(group.columns, axis=1).sum(axis=1)
        corr[index] = stats.pearsonr(strategy, ptf).statistic
corr_check = pd.Series(corr, dtype="float").rename('value').to_frame().rename_axis('strategy').assign(limit=correlation_limit) * 100

var_check = var_check.assign(passed=var_check['value'] <= var_check['limit'])
standalone_var_check = standalone_var_check.assign(
    passed=standalone_var_check['value'] <= standalone_var_check['limit'])
tail_risk_check = tail_risk_check.assign(
    passed=tail_risk_check['value'] <= tail_risk_check['limit']
)
corr_check = corr_check.assign(passed=corr_check['value'] <= corr_check['limit'])
strategy_check = strategy_check.assign(passed=strategy_check['count'] <= strategy_check['limit'])
position_check = position_check.assign(passed=position_check['count'] <= position_check['limit'])

C:\dev\projects\menashe\api\risk\var.py:172: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ).sum()
C:\dev\projects\menashe\api\risk\var.py:172: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ).sum()


In [177]:
tail_risk_check

value  limit  passed
daily  2022-06-13 1.7046 3.5000    True
weekly 2023-03-17 3.0902 3.5000    True

In [178]:
def highlight_pass(s, c1='', c2=''):
    return np.where(s == True, f'background-color: {c1}', f'background-color: {c2}')

In [182]:
var_check_fmt = var_check.style.apply(highlight_pass, c1='lime', c2='red', axis=0, subset='passed').format(partial(millify, decimal=2, pct=None))
standalone_var_check_fmt = standalone_var_check.style.apply(highlight_pass, c1='lime', c2='red', axis=0, subset='passed').format(partial(millify, decimal=2, pct=None))
tail_risk_check_fmt = tail_risk_check.style.apply(highlight_pass, c1='lime', c2='red', axis=0, subset='passed').format(partial(millify, decimal=2, pct=None)).format_index("{:%d%b%Y}", level=1)
tail_risk_by_strategy_fmt = tail_risk_by_strategy.style.format(partial(millify, decimal=2, pct=None)).background_gradient(cmap='RdYlGn', axis=0)
weekly_marginal_tail_risk_by_strat_fmt = weekly_marginal_tail_risk_by_strat.style.format_index("{:%d%b%Y}", level=1).format(partial(millify, decimal=2, pct=None)).background_gradient(cmap='RdYlGn')
corr_check_fmt = corr_check.style.apply(highlight_pass, c1='lime', c2='red', axis=0, subset='passed').format(partial(millify, decimal=2, pct=None))
strategy_check_fmt = strategy_check.style.apply(highlight_pass, c1='lime', c2='red', axis=0, subset='passed').format(partial(millify, decimal=2, pct=None))
position_check_fmt = position_check.style.apply(highlight_pass, c1='lime', c2='red', axis=0, subset='passed').format(partial(millify, decimal=2, pct=None))

In [183]:
tail_risk_check_fmt

In [184]:
html = '<h2 style="font-family: Century Gothic, sans-serif;">VaR</h2>'
html = html + df_to_html(var_check_fmt)

html = html + '<h2 style="font-family: Century Gothic, sans-serif;">Standalone VaR</h2>'
html = html + df_to_html(standalone_var_check_fmt)

html = html + '<h2 style="font-family: Century Gothic, sans-serif;">Tail Risk</h2>'
html = html + df_to_html(tail_risk_check_fmt) + '</br>'
html = html + '<h3 style="font-family: Century Gothic, sans-serif;">Weekly Tail Risk by Strategy</h3>'
html = html + df_to_html(tail_risk_by_strategy_fmt)
html = html + '<h3 style="font-family: Century Gothic, sans-serif;">Weekly Marginal Tail Risk by Strategy</h3>'
html = html + df_to_html(weekly_marginal_tail_risk_by_strat_fmt)

if corr_check.empty:
    pass
else:
    html = html + '<h2 style="font-family: Century Gothic, sans-serif;">Correlation</h2>'
    html = html + df_to_html(corr_check_fmt)

html = html + '<h2 style="font-family: Century Gothic, sans-serif;">Strategies per Category</h2>'
html = html + df_to_html(strategy_check_fmt)

html = html + '<h2 style="font-family: Century Gothic, sans-serif;">Positions per Strategy</h2>'
html = html + df_to_html(position_check_fmt)

In [185]:
send_email = 'test'

In [186]:
if send_email:
    to_email_ = to_email if send_email is True else to_email_ext if send_email == 'ext' else to_email_test

# Send Email
    message = MIMEMultipart("alternative")
    message["Subject"] = f'INTERNAL RISK REPORT - {dv.date: %d%b%y}'
    message["From"] = from_email
    message["To"] = ','.join(to_email_)
    html_ = MIMEText(html, "html")
    message.attach(html_)
    smtp = smtplib.SMTP(smtp_server)
    smtp.sendmail(from_email, to_email_, message.as_string())

print('SUCCESS 2')

SUCCESS 2


In [145]:
to_email if send_email is True else to_email_ext if send_email == 'ext' else to_email_test

['giovambattista.perciaccante@brevanhoward.com', 'gioxc@hotmail.it']